# Store Sales

##### Goal: Explore data and decide what makes a store successful at sales.

The data used in this project were taken from the 
[Rossmann Store Sales Kaggle](https://www.kaggle.com/c/rossmann-store-sales) competition.

## 1. Frame the problem

What is your view of the problem?<br>
What questions can you ask that might get to addressiing the problem?

## 2. The Data

What data do you need? <br>
Where are these data? <br>
How can you get the data? <br>
What does the data look like? <br>
Does the data need cleaning?

#### We have our data in csv files.  Let's load the data and look at it.

In [1]:
#package with DataFrames for viewing/manipulating the data
import pandas as pd

# data visualization
import matplotlib.pyplot as plt
# advanced vizs
from bokeh.plotting import figure, output_file, show

%matplotlib inline

In [2]:
# importing sales data to learn
sales = pd.read_csv("./data/sales.csv", 
                    parse_dates = True, low_memory = False, index_col = 'Date')

# additional store data
stores = pd.read_csv("./data/stores.csv", 
                    low_memory = False)

### Exploratory Data Analysis
##### Understanding and cleaning the data
Here we go through the sale and store data, handle missing values and create new features for further analysis.

In [3]:
# first glance at the sale set
print("In total there are {0} rows and {1} columns: ".format(sales.shape[0], sales.shape[1]))
sales.head(5).append(sales.tail(5))

In total there are 1017209 rows and 8 columns: 


,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
Date,,,,,,,,
2015-07-31,1,5,5263,555,1,1,0,1
2015-07-31,2,5,6064,625,1,1,0,1
2015-07-31,3,5,8314,821,1,1,0,1
2015-07-31,4,5,13995,1498,1,1,0,1
2015-07-31,5,5,4822,559,1,1,0,1
2013-01-01,1111,2,0,0,0,0,a,1
2013-01-01,1112,2,0,0,0,0,a,1
2013-01-01,1113,2,0,0,0,0,a,1
2013-01-01,1114,2,0,0,0,0,a,1


##### Short description of what we're seeing
1017209 rows, 8 columns
<ul>
<li>Sales: the turnover for any given day (target variable).</li>
<li>Customers: the number of customers on a given day.</li>
<li>Open: an indicator for whether the store was open: 0 = closed, 1 = open.</li>
<li>Promo: indicates whether a store is running a promo on that day.</li>
<li>StateHoliday: indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays.</li>
<li>SchoolHoliday: indicates if the (Store, Date) was affected by the closure of public schools.</li>
</ul>

We are dealing with time series data so it will probably serve us to extract dates for further analysis. We also have two likely correlated vaiables (i.e., number of sales and number of customers) in the dataset, which can be combined into a new feature (SalesPerCustomer).

In [4]:
# data extraction
sales['Year'] = sales.index.year
sales['Month'] = sales.index.month
sales['Day'] = sales.index.day
sales['WeekOfYear'] = sales.index.weekofyear

# adding new variable
sales['SalePerCustomer'] = sales['Sales']/sales['Customers']
sales['SalePerCustomer'].describe()

count    844340.000000
mean          9.493619
std           2.197494
min           0.000000
25%           7.895563
50%           9.250000
75%          10.899729
max          64.957854
Name: SalePerCustomer, dtype: float64

#### What the table above tells us is that on average customers spend about &dollar;9.50 per day. <br>However, the min value indicates there are days with Sales equal to zero.

Why are there days when there are no sales?  Are the stores closed?  Or just unable to make sales on certain days?  

Let's check.

#### Missing values¶
##### Closed stores and zero sales stores

In [5]:
# closed stores
sales[(sales.Open == 0) & (sales.Sales == 0)].head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,WeekOfYear,SalePerCustomer
Date,,,,,,,,,,,,,
2015-07-31,292,5,0,0,0,1,0,1,2015,7,31,31,NaN
2015-07-31,876,5,0,0,0,1,0,1,2015,7,31,31,NaN
2015-07-30,292,4,0,0,0,1,0,1,2015,7,30,31,NaN
2015-07-30,876,4,0,0,0,1,0,1,2015,7,30,31,NaN
2015-07-29,292,3,0,0,0,1,0,1,2015,7,29,31,NaN


<font color='blue'>Note: A value of 0 in the Open column indicates False and 1 indictes True.</font>

So 0 under Open means the store is closed. 

How many closed stores are in the data?

In [6]:
print("There are {} closed stores in the data.".format(sales[(sales.Open == 0) & (sales.Sales == 0)].shape[0]))
print("This is {}% of the observations".format( round(sales[(sales.Open == 0) & (sales.Sales == 0)].shape[0]/sales.shape[0]*100.0,2) ))

There are 172817 closed stores in the data.
This is 16.99% of the observations


To avoid any biased forecasts we will drop these values.

What about opened stores with zero sales?

In [7]:
# opened stores
sales[(sales.Open == 1) & (sales.Sales == 0)].head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,WeekOfYear,SalePerCustomer
Date,,,,,,,,,,,,,
2015-05-15,971,5,0,0,1,0,0,1,2015,5,15,20,NaN
2015-03-26,674,4,0,0,1,0,0,0,2015,3,26,13,NaN
2015-02-05,699,4,0,0,1,1,0,0,2015,2,5,6,NaN
2014-10-01,708,3,0,0,1,1,0,0,2014,10,1,40,NaN
2014-09-22,357,1,0,0,1,0,0,0,2014,9,22,39,NaN


In [8]:
print("There are {} open stores with no sales in the data.".format(sales[(sales.Open == 1) & (sales.Sales == 0)].shape[0]))

There are 54 open stores with no sales in the data.


Interestingly enough, there are opened store with no sales on working days. There're only 54 days in the data, so we can assume that there were external factors involved, for example power outage/system malfunction.

In [9]:
print("Closed stores and days which didn't have any sales won't be counted into the forecasts.")
sales = sales[(sales["Open"] != 0) & (sales['Sales'] != 0)]

print("In total {} stores remain in your data set".format(sales.shape[0]))

Closed stores and days which didn't have any sales won't be counted into the forecasts.
In total 844338 stores remain in your data set


We've removed the bad, dirty data from sales.

#### Now let's explore the store data

Does this data set have anything that we can add to the sales data to enhance our analysis?

In [10]:
# additional information about the stores
stores.head(10)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
5,6,a,a,310.0,12.0,2013.0,0,NaN,NaN,NaN
6,7,a,c,24000.0,4.0,2013.0,0,NaN,NaN,NaN
7,8,a,a,7520.0,10.0,2014.0,0,NaN,NaN,NaN
8,9,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN
9,10,a,a,3160.0,9.0,2009.0,0,NaN,NaN,NaN


##### Short description of what we're seeing
<ul>
<li>Store: a unique Id for each store</li>
<li>StoreType: differentiates between 4 different store models: a, b, c, d</li>
<li>Assortment: describes an assortment level: a = basic, b = extra, c = extended</li>
<li>CompetitionDistance: distance in meters to the nearest competitor store</li>
<li>CompetitionOpenSince[Month/Year]: gives the approximate year and month of the time the nearest competitor was opened</li>
<li>Promo2: Promo2 is a continuing a promotion for some stores: 0 = store is not participating, 1 = store is participating</li>
<li>Promo2Since[Year/Week]: describes the year and calendar week when the store started participating in Promo2</li>
<li>PromoInterval: describes the consecutive intervals Promo2 is started, naming the months the promotion is started. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store</li>
</ul>

#### NaN

Above we see some null or NaN (Not a Number) values.  We need to take care of this.

In [11]:
# how  many fields have missing values?
stores.isnull().sum()

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

We have few variables with missing values that we need to deal with. Let's start with the CompetitionDistance.

In [12]:
# missing values in CompetitionDistance
stores[pd.isnull(stores.CompetitionDistance)]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
290,291,d,a,NaN,NaN,NaN,0,NaN,NaN,NaN
621,622,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN
878,879,d,a,NaN,NaN,NaN,1,5.0,2013.0,"Feb,May,Aug,Nov"


#### How do we want to handle the missing data?

Apperently this information is simply missing from the data. No particular pattern observed. In this case, it makes sense to replace NaN with the median values.

In [13]:
# fill NaN with a median value (skewed distribuion)
stores['CompetitionDistance'].fillna(stores['CompetitionDistance'].median(), inplace = True)
stores[stores.index == 290]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
290,291,d,a,2325.0,NaN,NaN,0,NaN,NaN,NaN


We printed one line from the DataFrame to ensure that CompetitionDistance is updated to the median and is no longer NaN.

We got rid of one column of NaNs.  But still have others.

#### Continuing further with the missing data.... 
What about Promo2SinceWeek? How does it relate to Promo2?

In [14]:
# no promo = no information about the promo?
_ = stores[pd.isnull(stores.Promo2SinceWeek)]
_.head(10)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
5,6,a,a,310.0,12.0,2013.0,0,NaN,NaN,NaN
6,7,a,c,24000.0,4.0,2013.0,0,NaN,NaN,NaN
7,8,a,a,7520.0,10.0,2014.0,0,NaN,NaN,NaN
8,9,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN
9,10,a,a,3160.0,9.0,2009.0,0,NaN,NaN,NaN
15,16,a,c,3270.0,NaN,NaN,0,NaN,NaN,NaN
22,23,d,a,4060.0,8.0,2005.0,0,NaN,NaN,NaN


In [15]:
_[_.Promo2 != 0].shape

(0, 10)

We see that there are only NaN entries when there is no Promo2 (<em>Note: as before an entry of 0 means False, no Promo and 1 means True there is a Promo</em>). 

We can replace the Promo2Since values by zeros. 

The same is true for tha variables deducted from the competition: CompetitionOpenSinceMonth and CompetitionOpenSinceYear.

In [16]:
# replace NA's by 0
stores.fillna(0, inplace = True)

#### Now that we've cleaned up the data, let's combine our sales and stores data sets to enhance our analysis

In [17]:
print("Joining sales set with additional store information.")

# by specifying inner join we make sure that only those observations 
# that are present in both sales and stores sets are merged together
sales_stores = pd.merge(sales, stores, how = 'inner', on = 'Store')

print("In total: ", sales_stores.shape)
sales_stores.head()

Joining sales set with additional store information.
In total:  (844338, 22)


,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,...,SalePerCustomer,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,5263,555,1,1,0,1,2015,7,...,9.482883,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
1,1,4,5020,546,1,1,0,1,2015,7,...,9.194139,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
2,1,3,4782,523,1,1,0,1,2015,7,...,9.143403,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
3,1,2,5011,560,1,1,0,1,2015,7,...,8.948214,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
4,1,1,6102,612,1,1,0,1,2015,7,...,9.970588,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0


#### Store types

Below we will closely look at different levels of StoreType and how the main metric Sales is distributed among them.

In [18]:
sales_stores.groupby('StoreType')['Sales'].describe()

,count,mean,std,min,25%,50%,75%,max
StoreType,,,,,,,,
a,457042.0,6925.697986,3277.351589,46.0,4695.25,6285.0,8406.00,41551.0
b,15560.0,10233.380141,5155.729868,1252.0,6345.75,9130.0,13184.25,38722.0
c,112968.0,6933.126425,2896.958579,133.0,4916.00,6408.0,8349.25,31448.0
d,258768.0,6822.300064,2556.401455,538.0,5050.00,6395.0,8123.25,38037.0


StoreType B has the highest average of Sales among all others, however we have much less data for it. 

So let's print an overall sum of Sales and Customers to see which StoreType is the most selling and crowded one:

In [19]:
sales_stores.groupby('StoreType')['Customers', 'Sales'].sum()

,Customers,Sales
StoreType,,
a,363541431,3165334859
b,31465616,159231395
c,92129705,783221426
d,156904995,1765392943


Clearly stores of type A is the most selling and crowded type. StoreType D goes on the second place in both Sales and Customers. 

What about over time? What affects sales performance over time?

#### Effect of Promotions

Let's look to see if promotions seem to be  successful at increasing sales


Breaking the down the steps.

Steps:
<ol>
<li>Separate sales by whether they occurred during Promo</li>
<li>Separate sales by StoreType</li>
<li>Look at mean sales by month (i.e., looking at monthly sale patterns with and without promotions)</li>
</ol>

Separating by Promo True or False

In [20]:
sales_promo_true = sales_stores[sales_stores["Promo"]==1][["StoreType","Customers","Month","Sales"]] #promotion 
sales_promo_false = sales_stores[sales_stores["Promo"]==0][["StoreType","Customers","Month","Sales"]] #no promotion 

Separating by StoreType (a, b, c, or d) and Aggregating Sales by month

In [21]:
sales_promo_true = sales_promo_true.sort_values(["StoreType", "Month"]).groupby(["StoreType", "Month"]).agg({'Sales':'mean'})
sales_promo_true = sales_promo_true.reset_index().set_index("StoreType")
sales_promo_false = sales_promo_false.sort_values(["StoreType", "Month"]).groupby(["StoreType", "Month"]).agg({'Sales':'mean'})
sales_promo_false = sales_promo_false.reset_index().set_index("StoreType")

In [22]:
sales_promo_false

,Month,Sales
StoreType,,
a,1,5497.798096
a,2,5659.586170
a,3,5648.900700
a,4,5759.521846
a,5,6100.573560
a,6,5749.688821
a,7,5646.143726
a,8,5587.428817
a,9,5463.014076


In [23]:
x = sales_promo_true.loc["a"].Month.tolist()
y = sales_promo_true.loc["a"].Sales.tolist()

In [24]:
x_a = sales_promo_false.loc["a"].Month.tolist()
y_a = sales_promo_false.loc["a"].Sales.tolist()

In [25]:
x_b = sales_promo_false.loc["b"].Month.tolist()
y_b = sales_promo_false.loc["b"].Sales.tolist()

In [26]:
x_c = sales_promo_false.loc["c"].Month.tolist()
y_c = sales_promo_false.loc["c"].Sales.tolist()

In [27]:
x_d = sales_promo_false.loc["d"].Month.tolist()
y_d = sales_promo_false.loc["d"].Sales.tolist()

The code below should create a plot (ave_sale_trends.html) using bokeh.

The Bokeh Plot should automatically open in a new window.

In [28]:
# sales trends
output_file("ave_sale_trends.html")

p = figure()

# add a line renderer
p.multi_line([x_a,x_b,x_c,x_d],[y_a,y_b,y_c,y_d], color=["purple", "firebrick", "blue", "orange"], line_width=2)

show(p)

In [29]:
y_a

[5497.798095533019,
 5659.586170001981,
 5648.9007001118525,
 5759.5218458933105,
 6100.573559866447,
 5749.688820682328,
 5646.1437264561,
 5587.4288171787875,
 5463.014075887393,
 5776.346458097901,
 6093.620624209195,
 7120.138211382114]

In [30]:
y_b

[8567.865687303252,
 9433.128233970754,
 9554.377777777778,
 9479.557800224467,
 10252.717139852786,
 9854.835616438357,
 9503.257777777777,
 9084.234836702955,
 9150.350318471337,
 9488.328682170542,
 10010.293103448275,
 10528.754689754689]

In [31]:
a = sales_stores[sales_stores["Promo"]==1][["StoreType","Customers","Month","Sales"]]
a = a.sort_values(["StoreType", "Month"]).groupby(["StoreType", "Month"])
a.get_group(('a',1)).mean()

Customers     848.128576
Month           1.000000
Sales        7842.248758
dtype: float64